- Put your password for sudo in `./sudo_password.txt`, since it will be needed for running CICFlowMeter.
- PCAP from original CICIDS2017 must be reordered by `reordercap`

In [1]:
PCAP_INPUT = "/mnt/extra/datasets/pcap/wednesday-friday-24-april-2025.pcap"
FLOW_INPUT_1 = "/mnt/extra/datasets/fixed-2/wednesday.csv"
FLOW_INPUT_2 = "/mnt/extra/datasets/fixed-2/friday.csv"
OUTPUT_FOLDER = "./ExtractFlowsOutput"
EXTRACT_FLOWS_OUTPUT_FOLDER = OUTPUT_FOLDER

TEST_PERCENTAGE = 0.2

GRACE_PERCENTAGE = 0
TEST_PERCENTAGE += GRACE_PERCENTAGE

In [1]:
import os

def extract_flows_from_pcap(filepath: str, output_name: str):
    basename = os.path.basename(filepath)
    flow_filename = f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{output_name}_flow.csv"
    index_filename = f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{output_name}_index.txt"

    # print(f'sudo -S java -Djava.library.path="./Tools/jnetpcap-1.4.r1500" -jar ./Tools/CICFlowMeterV3-0.0.4-SNAPSHOT.jar {filepath} {EXTRACT_FLOWS_OUTPUT_FOLDER} < ./sudo_password.txt')

    os.system(f'sudo -S java -Djava.library.path="./Tools/jnetpcap-1.4.r1500" -jar ./Tools/CICFlowMeterV3-0.0.4-SNAPSHOT.jar {filepath} {EXTRACT_FLOWS_OUTPUT_FOLDER} < ./sudo_password.txt')
    
    os.rename(f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{basename}_Flow.csv", flow_filename)
    os.rename(f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{basename}_Flow.csv_index.txt", index_filename)

    return flow_filename, index_filename

In [4]:
extract_flows_from_pcap("/mnt/extra/datasets/pcap/wedfri_test.pcap", "wedfri_testing_again")

[sudo] password for meervix: 

cic.cs.unb.ca.ifm.Cmd You select: /mnt/extra/datasets/pcap/wedfri_test.pcap
cic.cs.unb.ca.ifm.Cmd Out folder: ./ExtractFlowsOutput
cic.cs.unb.ca.ifm.Cmd CICFlowMeter received 1 pcap file
Working on... wedfri_test.pcap
wedfri_test.pcap is done. total 128072 flows 
Packet stats: Total=1725174,Valid=1725174,Discarded=0
-------------------------------------------------------------------------------


('./ExtractFlowsOutput/wedfri_testing_again_flow.csv',
 './ExtractFlowsOutput/wedfri_testing_again_index.txt')

# Pre-Process Dataset
- Adjust labels to make it compatible with research scope
- Remove rows that makes mapping between packets and flows hard

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
distinguishing_keys = [
    'Src IP',
    'Src Port',
    'Dst IP',
    'Dst Port',
    'Timestamp',
]

In [4]:
flow_input_1 = pd.read_csv(FLOW_INPUT_1, parse_dates=['Timestamp'])
flow_input_2 = pd.read_csv(FLOW_INPUT_2, parse_dates=['Timestamp'])

In [5]:
compiled = pd.concat([flow_input_1, flow_input_2]).reset_index().drop(['id'], axis=1)
compiled

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.644583,187889.000,0.000000,187889,187889,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.322291,5.322291,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,187889,BENIGN,-1
1,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.652010,187758.000,0.000000,187758,187758,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.326005,5.326005,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,187758,BENIGN,-1
2,2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.532857,189882.000,0.000000,189882,189882,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.266429,5.266429,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,189882,BENIGN,-1
3,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.519838,190117.000,0.000000,190117,190117,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.259919,5.259919,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,190117,BENIGN,-1
4,4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.604285,188603.000,0.000000,188603,188603,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.302143,5.302143,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,188603,BENIGN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [6]:
compiled['Label'].value_counts(dropna=False)

Label
BENIGN                          607664
Portscan                        159066
DoS Hulk                        158468
DDoS                             95144
DoS GoldenEye                     7567
Botnet - Attempted                4067
DoS Slowloris                     3859
DoS Slowhttptest - Attempted      3368
DoS Slowloris - Attempted         1847
DoS Slowhttptest                  1740
Botnet                             736
DoS Hulk - Attempted               581
DoS GoldenEye - Attempted           80
Heartbleed                          11
Name: count, dtype: int64

In [8]:
compiled[compiled['Label'] == 'DoS Slowloris - Attempted'].sort_values(by='Timestamp')

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
25347,25347,172.16.0.1-192.168.10.50-53418-80-6,172.16.0.1,53418,192.168.10.50,80,6,2017-07-05 12:48:46.211817,573660,5,4,231,483,231,0,46.200000,103.306341,483,0,120.750000,241.500000,1244.639682,15.688735,71707.500000,1.944356e+05,552804,2,573660,143415.000000,2.730363e+05,552920,1457,572756,190918.666667,3.229729e+05,563820,2,1,1,0,0,1,0,156,136,8.715964,6.972771,0,483,79.333333,169.560461,2.875075e+04,2,2,1,2,7,0,0,0,0.800000,79.333333,46.200000,120.750000,0,0,0,0,0,0,0,25,0,53,29200,235,1,20,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,573660,DoS Slowloris - Attempted,5
25348,25348,172.16.0.1-192.168.10.50-53420-80-6,172.16.0.1,53420,192.168.10.50,80,6,2017-07-05 12:48:46.220819,564248,5,4,231,483,231,0,46.200000,103.306341,483,0,120.750000,241.500000,1265.401029,15.950433,70531.000000,1.945252e+05,551868,9,564248,141062.000000,2.739770e+05,551954,240,563645,187881.666667,3.246484e+05,562753,9,1,1,0,0,1,0,156,136,8.861352,7.089081,0,483,79.333333,169.560461,2.875075e+04,2,2,1,2,7,0,0,0,0.800000,79.333333,46.200000,120.750000,0,0,0,0,0,0,0,25,0,53,29200,235,1,20,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,564248,DoS Slowloris - Attempted,5
25349,25349,172.16.0.1-192.168.10.50-53422-80-6,172.16.0.1,53422,192.168.10.50,80,6,2017-07-05 12:48:46.221774,561721,5,4,231,483,231,0,46.200000,103.306341,483,0,120.750000,241.500000,1271.093657,16.022189,70215.125000,1.943969e+05,551258,1,561721,140430.250000,2.739465e+05,551294,4,561128,187042.666667,3.234756e+05,560560,1,1,1,0,0,1,0,156,136,8.901216,7.120973,0,483,79.333333,169.560461,2.875075e+04,2,2,1,2,7,0,0,0,0.800000,79.333333,46.200000,120.750000,0,0,0,0,0,0,0,25,0,53,29200,235,1,20,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,561721,DoS Slowloris - Attempted,5
25351,25351,172.16.0.1-192.168.10.50-53424-80-6,172.16.0.1,53424,192.168.10.50,80,6,2017-07-05 12:48:46.222126,555185,5,4,231,483,231,0,46.200000,103.306341,483,0,120.750000,241.500000,1286.057801,16.210813,69398.125000,1.941009e+05,549767,2,555185,138796.250000,2.740166e+05,549816,588,554458,184819.333333,3.186851e+05,552804,2,1,1,0,0,1,0,156,136,9.006007,7.204806,0,483,79.333333,169.560461,2.875075e+04,2,2,1,2,7,0,0,0,0.800000,79.333333,46.200000,120.750000,0,0,0,0,0,0,0,25,0,53,29200,235,1,20,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,555185,DoS Slowloris - Attempted,5
25350,25350,172.16.0.1-192.168.10.50-53426-80-6,172.16.0.1,53426,192.168.10.50,80,6,2017-07-05 12:48:46.222403,552099,5,4,231,483,231,0,46.200000,103.306341,483,0,120.750000,241.500000,1293.246320,16.301424,69012.375000,1.944475e+05,550245,4,552099,138024.750000,2.748650e+05,550322,230,551487,183829.000000,3.176697e+05,550642,4,1,1

In [9]:
compiled[compiled['Label'] == 'DoS Slowloris'].sort_values(by='Timestamp')

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
9898,9898,172.16.0.1-192.168.10.50-53734-80-6,172.16.0.1,53734,192.168.10.50,80,6,2017-07-05 12:48:54.539220,100053357,5,4,247,0,231,0,49.4,101.596260,0,0,0.0,0.00000,2.468683,0.089952,1.250667e+07,3.535241e+07,99999399,39,100053173,2.501329e+07,4.999077e+07,99999438,880,100053229,3.335108e+07,5.771930e+07,99999583,4549,3,0,0,0,0,0,168,136,0.049973,0.039979,0,231,27.444444,76.411896,5838.777778,0,2,0,3,8,0,0,0,0.800000,27.444444,49.4,0.0,0,0,0,0,0,0,0,27,0,0,29200,235,3,32,53774.0,0.0,53774,53774,99999399.0,0.0,99999399,99999399,-1,-1,100053357,DoS Slowloris,-1
9899,9899,172.16.0.1-192.168.10.50-53736-80-6,172.16.0.1,53736,192.168.10.50,80,6,2017-07-05 12:48:54.543862,100048827,5,4,247,0,231,0,49.4,101.596260,0,0,0.0,0.00000,2.468795,0.089956,1.250610e+07,3.535273e+07,99999648,30,100048790,2.501220e+07,4.999166e+07,99999678,267,100048748,3.334958e+07,5.772069e+07,99999685,827,3,0,0,0,0,0,168,136,0.049976,0.039980,0,231,27.444444,76.411896,5838.777778,0,2,0,3,8,0,0,0,0.800000,27.444444,49.4,0.0,0,0,0,0,0,0,0,27,0,0,29200,235,3,32,49142.0,0.0,49142,49142,99999648.0,0.0,99999648,99999648,-1,-1,100048827,DoS Slowloris,-1
9901,9901,172.16.0.1-192.168.10.50-53738-80-6,172.16.0.1,53738,192.168.10.50,80,6,2017-07-05 12:48:54.544760,100047997,5,4,247,0,231,0,49.4,101.596260,0,0,0.0,0.00000,2.468815,0.089957,1.250600e+07,3.535276e+07,99999614,53,100047896,2.501197e+07,4.999180e+07,99999671,266,100047944,3.334931e+07,5.772094e+07,99999715,859,3,0,0,0,0,0,168,136,0.049976,0.039981,0,231,27.444444,76.411896,5838.777778,0,2,0,3,8,0,0,0,0.800000,27.444444,49.4,0.0,0,0,0,0,0,0,0,27,0,0,29200,235,3,32,48282.0,0.0,48282,48282,99999614.0,0.0,99999614,99999614,-1,-1,100047997,DoS Slowloris,-1
9900,9900,172.16.0.1-192.168.10.50-53740-80-6,172.16.0.1,53740,192.168.10.50,80,6,2017-07-05 12:48:54.545654,100047104,5,4,247,0,231,0,49.4,101.596260,0,0,0.0,0.00000,2.468837,0.089958,1.250589e+07,3.535281e+07,99999622,40,100047010,2.501175e+07,4.999196e+07,99999679,589,100047038,3.334901e+07,5.772121e+07,99999716,2165,3,0,0,0,0,0,168,136,0.049976,0.039981,0,231,27.444444,76.411896,5838.777778,0,2,0,3,8,0,0,0,0.800000,27.444444,49.4,0.0,0,0,0,0,0,0,0,27,0,0,29200,235,3,32,47388.0,0.0,47388,47388,99999622.0,0.0,99999622,99999622,-1,-1,100047104,DoS Slowloris,-1
9902,9902,172.16.0.1-192.168.10.50-53742-80-6,172.16.0.1,53742,192.168.10.50,80,6,2017-07-05 12:48:54.547893,100044862,5,4,247,0,231,0,49.4,101.596260,0,0,0.0,0.00000,2.468892,0.089960,1.250561e+07,3.535293e+07,99999631,55,100044791,2.501120e+07,4.999235e+07,99999722,493,100044806,3.334827e+07,5.772184e+07,99999702,1268,3,0,

In [10]:
LABEL_MAPPING = {
    'BENIGN': 'Benign',
    'Portscan': 'Portscan',
    'DoS Slowloris': 'Slowloris',
    'DoS Slowloris - Attempted': 'Slowloris',
}

In [11]:
compiled['Label'] = compiled['Label'].map(LABEL_MAPPING)
compiled = compiled.dropna(subset=['Label'])
compiled['Label'].value_counts(dropna=False)

Label
Benign       607664
Portscan     159066
Slowloris      5706
Name: count, dtype: int64

In [12]:
compiled_keys = compiled[distinguishing_keys]
has_duplicate = (compiled_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

compiled_unique = compiled[~compiled_keys.duplicated(keep=False)]
compiled_unique

Has Duplicates? False


,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.644583,187889.000,0.000000,187889,187889,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.322291,5.322291,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,187889,Benign,-1
1,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.652010,187758.000,0.000000,187758,187758,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.326005,5.326005,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,187758,Benign,-1
2,2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.532857,189882.000,0.000000,189882,189882,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.266429,5.266429,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,189882,Benign,-1
3,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.519838,190117.000,0.000000,190117,190117,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.259919,5.259919,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,190117,Benign,-1
4,4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.604285,188603.000,0.000000,188603,188603,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.302143,5.302143,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,188603,Benign,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [13]:
compiled_unique.to_csv(f"./ExtractFlowsOutput/wedfri_preprocess_1.csv", index=False)

In [5]:
compiled = pd.read_csv(f"./ExtractFlowsOutput/wedfri_preprocess_1.csv", parse_dates=['Timestamp'])
compiled

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.644583,187889.000,0.000000,187889,187889,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.322291,5.322291,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,187889,Benign,-1
1,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.652010,187758.000,0.000000,187758,187758,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.326005,5.326005,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,187758,Benign,-1
2,2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.532857,189882.000,0.000000,189882,189882,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.266429,5.266429,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,189882,Benign,-1
3,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.519838,190117.000,0.000000,190117,190117,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.259919,5.259919,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,190117,Benign,-1
4,4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.604285,188603.000,0.000000,188603,188603,0,0.00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,1,32,20,5.302143,5.302143,0,0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,188603,Benign,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [6]:
compiled['Label'].value_counts(dropna=False)

Label
Benign       607664
Portscan     159066
Slowloris      5706
Name: count, dtype: int64

# Run CICFlowMeter on PCAP Dataset
- Get features from current version of CICFlowMeter (e.g. to fix bugs in features caused by old CICFlowMeter)

In [3]:
pcap_input = '/mnt/extra/datasets/pcap/downloading/Wednesday-workingHours-Reordered.pcap'
rerun_friday_flow_file, rerun_friday_index_file = extract_flows_from_pcap(pcap_input, f"wedfri_rerun_1")

sudo -S java -Djava.library.path="./Tools/jnetpcap-1.4.r1500" -jar ./Tools/CICFlowMeterV3-0.0.4-SNAPSHOT.jar /mnt/extra/datasets/pcap/downloading/Wednesday-workingHours-Reordered.pcap ./ExtractFlowsOutput < ./sudo_password.txt


TypeError: cannot unpack non-iterable NoneType object

In [19]:
rerun1_flow_file = f"{OUTPUT_FOLDER}/wednesday-friday-24-april-2025.pcap_Flow.csv"
rerun1_index_file = f"{OUTPUT_FOLDER}/wednesday-friday-24-april-2025.pcap_Flow.csv_index.txt"

In [20]:
flowmeter = pd.read_csv(rerun1_flow_file, parse_dates=['Timestamp'], date_format="mixed")
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,187889.000000,0.000000,187889.0,187889.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,187889,NeedManualLabel
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,187758.000000,0.000000,187758.0,187758.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,187758,NeedManualLabel
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,189882.000000,0.000000,189882.0,189882.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,189882,NeedManualLabel
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,190117.000000,0.000000,190117.0,190117.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,190117,NeedManualLabel
4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.604285,188603.000000,0.000000,188603.0,188603.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.302143,5.302143,0.0,0.0,

In [28]:
# flink_test[flink_test['Timestamp'] == '2017-07-05 13:10:11.354291']
flowmeter[flowmeter['Flow Duration'] == 72263.0]

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
815773,192.168.10.50-192.168.10.3-42910-389-6,192.168.10.50,42910,192.168.10.3,389,6,2017-07-07 20:02:38.387716,72263,8,4,1612.0,1264.0,403.0,0.0,201.5,215.412561,316.0,316.0,316.0,0.0,39799.067296,166.060086,6569.363636,14685.848966,41414.0,3.0,72263.0,10323.285714,17817.997865,41648.0,3.0,30565.0,10188.333333,17639.782775,30557.0,4.0,4,4,0,0,0,0,256,128,110.706724,55.353362,0.0,403.0,239.666667,180.851186,32707.151515,0,0,0,8,12,0,0,0,0.5,239.666667,201.5,316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,435,2078,4,4,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,4,2,6,126534055,NeedManualLabel


In [27]:
flink_test[flink_test['Flow Duration'] == 72263.0]

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
153910,154073,ce1c7f4d-e6c5-4711-b6e1-207f5dfcbe45,192.168.10.50,42910,192.168.10.3,389,6,2017-07-07 20:02:38.387716,72263.0,8,4,1612.0,1264.0,403.0,0.0,201.5,215.412561,316.0,316.0,316.0,0.0,39799.067296,166.060086,6569.363636,14685.848966,41414.0,3.0,72263.0,10323.285714,17817.997865,41648.0,3.0,30565.0,10188.333333,17639.782775,30557.0,4.0,4,4,0,0,0,0,256,128,110.706724,55.353362,0.0,403.0,239.666667,180.851186,32707.151515,0,0,0,8,12,0,0,0,0.5,239.666667,201.5,316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,435,2078,4,4,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4,2,6,0,Benign


In [12]:
flowmeter_keys = flowmeter[distinguishing_keys]
flowmeter = flowmeter[~flowmeter_keys.duplicated(keep=False)]
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,187889.000000,0.000000,187889.0,187889.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,187889,NeedManualLabel
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,187758.000000,0.000000,187758.0,187758.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,187758,NeedManualLabel
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,189882.000000,0.000000,189882.0,189882.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,189882,NeedManualLabel
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,190117.000000,0.000000,190117.0,190117.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,190117,NeedManualLabel
4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.604285,188603.000000,0.000000,188603.0,188603.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.302143,5.302143,0.0,0.0,

# Annotate Rerunned CSV using Dataset's CSV

In [13]:
compiled_keys = compiled[distinguishing_keys + ['Label']]
compiled[compiled_keys.duplicated(keep=False)]

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category


In [14]:
flowmeter_keys = flowmeter[distinguishing_keys]
flowmeter[flowmeter_keys.duplicated(keep=False)]

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label


In [15]:
left_merge = flowmeter_keys.reset_index().merge(compiled_keys, on=distinguishing_keys, how='left', indicator=True).set_index('index')
left_merge

,Src IP,Src Port,Dst IP,Dst Port,Timestamp,Label,_merge
index,,,,,,,
0,192.168.10.9,13793,52.197.242.182,443,2017-07-05 11:43:34.783702,Benign,both
1,192.168.10.9,13793,52.197.242.182,443,2017-07-05 11:43:35.484765,Benign,both
2,192.168.10.9,13794,54.65.28.113,443,2017-07-05 11:43:36.375217,Benign,both
3,192.168.10.9,13794,54.65.28.113,443,2017-07-05 11:43:37.075970,Benign,both
4,192.168.10.9,13796,52.197.242.182,443,2017-07-05 11:43:37.968708,Benign,both
...,...,...,...,...,...,...,...
1045639,192.168.10.5,51818,192.168.10.3,53,2017-07-05 18:20:02.014027,Benign,both
1045640,172.16.0.1,33248,192.168.10.50,9040,2017-07-07 17:54:38.857717,Portscan,both
1045641,172.16.0.1,44033,192.168.10.50,1272,2017-07-07 17:52:03.285040,Portscan,both


In [16]:
flowmeter_ignored = left_merge[left_merge['_merge'] == 'left_only']
flowmeter_ignored_idx = flowmeter_ignored.index
flowmeter_merged = flowmeter.drop(index=flowmeter_ignored_idx)

In [17]:
flowmeter_merged['Label'] = left_merge[left_merge['_merge'] == "both"]['Label']

In [18]:
flowmeter_merged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,187889.000000,0.000000,187889.0,187889.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,187889,Benign
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,187758.000000,0.000000,187758.0,187758.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,187758,Benign
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,189882.000000,0.000000,189882.0,189882.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,189882,Benign
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,190117.000000,0.000000,190117.0,190117.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,190117,Benign
4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.604285,188603.000000,0.000000,188603.0,188603.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.302143,5.302143,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0

In [19]:
flowmeter_merged['Label'].value_counts()

Label
Benign       606589
Portscan     159066
Slowloris      5706
Name: count, dtype: int64

In [21]:
flowmeter_merged.reset_index().to_csv(f"{OUTPUT_FOLDER}/wedfri_annotated.csv", index=False)

## Load Checkpoint

In [28]:
flowmeter = pd.read_csv(f"{OUTPUT_FOLDER}/wedfri_annotated.csv", index_col=0, parse_dates=['Timestamp'], date_format="mixed")
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,187889.000000,0.000000,187889.0,187889.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,187889,Benign
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,187758.000000,0.000000,187758.0,187758.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,187758,Benign
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,189882.000000,0.000000,189882.0,189882.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,189882,Benign
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,190117.000000,0.000000,190117.0,190117.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.00,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,190117,Benign
4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.604285,188603.000000,0.000000,188603.0,188603.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0000

# Train-Test Split
## Test Split

In [30]:
tagged = flowmeter[flowmeter['Label'].isin(['Portscan', 'Slowloris'])]
tagged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9593,172.16.0.1-192.168.10.50-53816-80-6,172.16.0.1,53816,192.168.10.50,80,6,2017-07-05 12:48:54.567542,17072865,4,3,239.0,0.0,231.0,0.0,59.75,114.228937,0.0,0.0,0.0,0.0,13.998822,0.410007,2.845478e+06,6.969245e+06,17071390.0,60.0,17072805.0,5.690935e+06,9.855842e+06,17071481.0,473.0,17072800.0,8536400.0,1.207038e+07,17071450.0,1350.0,2,0,0,0,0,0,136,104,0.234290,0.175717,0.0,231.0,34.142857,86.857025,7544.142857,0,2,0,2,6,0,0,0,0.75,34.142857,59.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,239.0,3.0,0.0,29200,28960,2,0,32,0,1415.0,0.0,1415.0,1415.0,17071390.0,0.0,17071390.0,17071390.0,-1,-1,0,0,0,17072865,Slowloris,2017-07-05 12:49:11.640407
9594,172.16.0.1-192.168.10.50-53840-80-6,172.16.0.1,53840,192.168.10.50,80,6,2017-07-05 12:48:54.570946,17069963,4,3,239.0,0.0,231.0,0.0,59.75,114.228937,0.0,0.0,0.0,0.0,14.001202,0.410077,2.844994e+06,6.968335e+06,17069049.0,58.0,17069905.0,5.689968e+06,9.854632e+06,17069117.0,343.0,17069847.0,8534923.5,1.206916e+07,17069107.0,740.0,2,0,0,0,0,0,136,104,0.234330,0.175747,0.0,231.0,34.142857,86.857025,7544.142857,0,2,0,2,6,0,0,0,0.75,34.142857,59.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,239.0,3.0,0.0,29200,28960,2,0,32,0,856.0,0.0,856.0,856.0,17069049.0,0.0,17069049.0,17069049.0,-1,-1,0,0,0,17069963,Slowloris,2017-07-05 12:49:11.640909
9595,172.16.0.1-192.168.10.50-53824-80-6,172.16.0.1,53824,192.168.10.50,80,6,2017-07-05 12:48:54.569148,17071303,4,3,239.0,0.0,231.0,0.0,59.75,114.228937,0.0,0.0,0.0,0.0,14.000103,0.410045,2.845217e+06,6.968788e+06,17070196.0,45.0,17071198.0,5.690399e+06,9.855232e+06,17070241.0,399.0,17071214.0,8535607.0,1.206988e+07,17070301.0,913.0,2,0,0,0,0,0,136,104,0.234311,0.175734,0.0,231.0,34.142857,86.857025,7544.142857,0,2,0,2,6,0,0,0,0.75,34.142857,59.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,239.0,3.0,0.0,29200,28960,2,0,32,0,1002.0,0.0,1002.0,1002.0,17070196.0,0.0,17070196.0,17070196.0,-1,-1,0,0,0,17071303,Slowloris,2017-07-05 12:49:11.640451
9596,172.16.0.1-192.168.10.50-53834-80-6,172.16.0.1,53834,192.168.10.50,80,6,2017-07-05 12:48:54.570290,17070583,4,3,239.0,0.0,231.0,0.0,59.75,114.228937,0.0,0.0,0.0,0.0,14.000693,0.410062,2.845097e+06,6.968642e+06,17069779.0,2.0,17070519.0,5.690173e+06,9.855115e+06,17069880.0,2.0,17070535.0,8535267.5,1.206971e+07,17069843.0,692.0,2,0,0,0,0,0,136,104,0.234321,0.175741,0.0,231.0,34.142857,86.857025,7544.142857,0,2,0,2,6,0,0,0,0.75,34.142857,59.75,0.0,0.0,0.0,0.0,0.0,0.0,0

In [31]:
untagged = flowmeter[~flowmeter['Label'].isin(['Portscan', 'Slowloris'])]
untagged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,Benign,2017-07-05 11:43:34.971591
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,Benign,2017-07-05 11:43:35.672523
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,Benign,2017-07-05 11:43:36.565099
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000

In [34]:
import numpy as np

min_time = np.min(flowmeter['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(flowmeter['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Whole flows are from \"{min_time}\" to \"{max_time}\"")

Whole flows are from "2017-07-05 08:42:42.084372" to "2017-07-07 17:02:40.977798"


In [35]:
min_time = np.min(tagged['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(tagged['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Tagged flows are from \"{min_time}\" to \"{max_time}\"")

Tagged flows are from "2017-07-05 09:48:46.211817" to "2017-07-07 15:23:53.256235"


In [39]:
slowloris_min_time = pd.Timestamp("2017-07-05 09:47:00") + pd.Timedelta(hours=3)
slowloris_max_time = pd.Timestamp("2017-07-05 10:11:00") + pd.Timedelta(hours=3)
firewall_on_min_time = pd.Timestamp("2017-07-07 13:55:00") + pd.Timedelta(hours=3)
firewall_on_max_time = pd.Timestamp("2017-07-07 14:36:00") + pd.Timedelta(hours=3)
firewall_off_min_time = pd.Timestamp("2017-07-07 14:51:00") + pd.Timedelta(hours=3)
firewall_off_max_time = pd.Timestamp("2017-07-07 15:30:00") + pd.Timedelta(hours=3)

In [40]:
slowloris = tagged[(slowloris_min_time <= tagged['Timestamp']) & (tagged['Timestamp'] <= slowloris_max_time)]
slowloris_untagged = untagged[(slowloris_min_time <= untagged['Timestamp']) & (untagged['Timestamp'] <= slowloris_max_time)]

In [41]:
firewall_on = tagged[(firewall_on_min_time <= tagged['Timestamp']) & (tagged['Timestamp'] <= firewall_on_max_time)]
firewall_on_untagged = untagged[(firewall_on_min_time <= untagged['Timestamp']) & (untagged['Timestamp'] <= firewall_on_max_time)]

In [42]:
firewall_off = tagged[(firewall_off_min_time <= tagged['Timestamp']) & (tagged['Timestamp'] <= firewall_off_max_time)]
firewall_off_untagged = untagged[(firewall_off_min_time <= untagged['Timestamp']) & (untagged['Timestamp'] <= firewall_off_max_time)]

In [43]:
len(slowloris) + len(firewall_on) + len(firewall_off) == tagged['Label'].value_counts(dropna=False)['Portscan'] + tagged['Label'].value_counts(dropna=False)['Slowloris']

True

In [44]:
benign_test = int(len(untagged) * TEST_PERCENTAGE)
slowloris_test = int(len(slowloris) * TEST_PERCENTAGE)
firewall_on_test = int(len(firewall_on) * TEST_PERCENTAGE)
firewall_off_test = int(len(firewall_off) * TEST_PERCENTAGE)

print("Test Dataset Numbers:")
print(f"Benign: {benign_test}")
print(f"Slowloris: {slowloris_test}")
print(f"Firewall On: {firewall_on_test}")
print(f"Firewall Off: {firewall_off_test}")

Test Dataset Numbers:
Benign: 121317
Slowloris: 1141
Firewall On: 68
Firewall Off: 31744


In [45]:
# Slowloris
slowloris_timestamps = slowloris.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(slowloris_timestamps)):
    if slowloris_timestamps[i-1] > slowloris_timestamps[i]:
        print("NOT SORTED")

cnt = -1
slowloris_test_df = pd.DataFrame()
slowloris_chosen_time = None
for tmp in slowloris_timestamps[::-1]:
    if cnt >= slowloris_test:
        break
    slowloris_test_df = slowloris[(tmp <= slowloris['Timestamp'])]
    slowloris_chosen_time = tmp
    cnt = len(slowloris_test_df)

slowloris_untagged_test_df = slowloris_untagged[(slowloris_chosen_time <= slowloris_untagged['Timestamp'])]

print(slowloris_test)
print(slowloris_chosen_time)
print(len(slowloris_test_df))
print(len(slowloris_untagged_test_df))

1141
2017-07-05 13:05:55.604911
1141
2973


In [46]:
# Firewall On
firewall_on_timestamps = firewall_on.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(firewall_on_timestamps)):
    if firewall_on_timestamps[i-1] > firewall_on_timestamps[i]:
        print("NOT SORTED")

cnt = -1
firewall_on_test_df = pd.DataFrame()
firewall_on_chosen_time = None
for tmp in firewall_on_timestamps[::-1]:
    if cnt >= firewall_on_test:
        break
    firewall_on_test_df = firewall_on[(tmp <= firewall_on['Timestamp'])]
    firewall_on_chosen_time = tmp
    cnt = len(firewall_on_test_df)

firewall_on_untagged_test_df = firewall_on_untagged[(firewall_on_chosen_time <= firewall_on_untagged['Timestamp'])]

print(firewall_on_test)
print(firewall_on_chosen_time)
print(len(firewall_on_test_df))
print(len(firewall_on_untagged_test_df))

68
2017-07-07 17:14:50.690809
68
13578


In [47]:
# Firewall Off
firewall_off_timestamps = firewall_off.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(firewall_off_timestamps)):
    if firewall_off_timestamps[i-1] > firewall_off_timestamps[i]:
        print("NOT SORTED")

tmp = firewall_off_timestamps[::-1][31590] # change here
firewall_off_test_df = firewall_off[(tmp <= firewall_off['Timestamp'])]
firewall_off_untagged_test_df = firewall_off_untagged[(tmp <= firewall_off_untagged['Timestamp'])]
print(tmp)
print(f"{len(firewall_off_test_df)}/{firewall_off_test}")

2017-07-07 17:55:45.861440
31744/31744


In [48]:
benign_left = benign_test - (len(slowloris_untagged_test_df) + len(firewall_on_untagged_test_df) + len(firewall_off_untagged_test_df))
benign_left

78582

In [63]:
df_test_partial_benign = pd.concat([
    slowloris_test_df,
    slowloris_untagged_test_df,
    firewall_on_test_df,
    firewall_on_untagged_test_df,
    firewall_off_test_df,
    firewall_off_untagged_test_df,
])
print("Is Duplicated? " + str(df_test_partial_benign.duplicated().any()))
df_test_partial_benign['Label'].value_counts()

Is Duplicated? False


Label
Benign       42735
Portscan     31812
Slowloris     1141
Name: count, dtype: int64

In [56]:
benign_timestamps = untagged.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(benign_timestamps)):
    if benign_timestamps[i-1] > benign_timestamps[i]:
        print("NOT SORTED")

In [80]:
df_test_partial_benign['Timestamp'].unique()

<DatetimeArray>
['2017-07-05 13:06:06.833561', '2017-07-05 13:06:06.833566',
 '2017-07-05 13:06:06.833595', '2017-07-05 13:06:06.833606',
 '2017-07-05 13:06:06.833601', '2017-07-05 13:06:06.833616',
 '2017-07-05 13:06:06.833693', '2017-07-05 13:06:06.833672',
 '2017-07-05 13:06:06.833727', '2017-07-05 13:06:06.833744',
 ...
 '2017-07-07 18:07:37.142931', '2017-07-07 18:25:37.558216',
 '2017-07-07 18:28:40.239316', '2017-07-07 18:27:46.420333',
 '2017-07-07 18:28:21.430851', '2017-07-07 18:27:45.328118',
 '2017-07-07 18:12:49.320592', '2017-07-07 17:58:23.667640',
 '2017-07-07 18:14:42.257470', '2017-07-07 18:10:20.530731']
Length: 75504, dtype: datetime64[ns]

In [96]:
tmp = benign_timestamps[::-1][118300]
benign_test_df = untagged[
    (tmp <= untagged['Timestamp'])
    & (~untagged['Timestamp'].isin(df_test_partial_benign['Timestamp'].unique()))
]
print(tmp)
print(f"{len(benign_test_df)}/{benign_left}")

2017-07-07 16:50:07.834862
78582/78582


In [97]:
slowloris_test_df['Label'].value_counts()

Label
Slowloris    1141
Name: count, dtype: int64

In [98]:
slowloris_untagged_test_df['Label'].value_counts()

Label
Benign    2973
Name: count, dtype: int64

In [99]:
firewall_on_test_df['Label'].value_counts()

Label
Portscan    68
Name: count, dtype: int64

In [100]:
firewall_on_untagged_test_df['Label'].value_counts()

Label
Benign    13578
Name: count, dtype: int64

In [101]:
firewall_off_test_df['Label'].value_counts()

Label
Portscan    31744
Name: count, dtype: int64

In [102]:
firewall_off_untagged_test_df['Label'].value_counts()

Label
Benign    26184
Name: count, dtype: int64

In [103]:
benign_test_df['Label'].value_counts()

Label
Benign    78582
Name: count, dtype: int64

In [108]:
df_test = pd.concat([
    df_test_partial_benign,
    benign_test_df,
])
print("Has Duplicates? " + str(df_test.duplicated().any()))
df_test

Has Duplicates? False


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
15431,172.16.0.1-192.168.10.50-54018-80-6,172.16.0.1,54018,192.168.10.50,80,6,2017-07-05 13:06:06.833561,41,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.951220e+05,48780.487805,41.000000,0.000000,41.0,41.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,24390.243902,24390.243902,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1032238673,Slowloris,2017-07-05 13:06:06.833602
15432,172.16.0.1-192.168.10.50-54126-80-6,172.16.0.1,54126,192.168.10.50,80,6,2017-07-05 13:06:06.833566,9,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,8.888889e+05,222222.222222,9.000000,0.000000,9.0,9.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,111111.111111,111111.111111,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1024233804,Slowloris,2017-07-05 13:06:06.833575
15433,172.16.0.1-192.168.10.50-54140-80-6,172.16.0.1,54140,192.168.10.50,80,6,2017-07-05 13:06:06.833595,29,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,2.758621e+05,68965.517241,29.000000,0.000000,29.0,29.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,34482.758621,34482.758621,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1023232173,Slowloris,2017-07-05 13:06:06.833624
15434,172.16.0.1-192.168.10.50-54178-80-6,172.16.0.1,54178,192.168.10.50,80,6,2017-07-05 13:06:06.833606,58,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.379310e+05,34482.758621,58.000000,0.000000,58.0,58.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,17241.379310,17241.379310,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1021233337,Slowloris,2017-07-05 13:06:06.833664
15435,172.16.0.1-192.168.10.50-54146-80-6,172.16.0.1,54146

In [109]:
df_test['Label'].value_counts(dropna=False)

Label
Benign       121317
Portscan      31812
Slowloris      1141
Name: count, dtype: int64

In [110]:
flowmeter['Label'].value_counts()

Label
Benign       606589
Portscan     159066
Slowloris      5706
Name: count, dtype: int64

In [112]:
df_test.reset_index().to_csv(f"{OUTPUT_FOLDER}/wedfri_annotated_test.csv", index=False)

In [113]:
df_train = flowmeter.drop(index=df_test.index)
df_train

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,Benign,2017-07-05 11:43:34.971591
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,Benign,2017-07-05 11:43:35.672523
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,Benign,2017-07-05 11:43:36.565099
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000

In [114]:
df_train.reset_index().to_csv(f"{OUTPUT_FOLDER}/wedfri_annotated_train.csv", index=False)

In [8]:
df_train = pd.read_csv(f"{OUTPUT_FOLDER}/wedfri_annotated_train.csv", parse_dates=['Timestamp'], date_format="mixed")
df_train

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
0,0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,Benign,2017-07-05 11:43:34.971591
1,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,Benign,2017-07-05 11:43:35.672523
2,2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,Benign,2017-07-05 11:43:36.565099
3,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,190117,Benign,2017-07-05 11:43:37.266

In [42]:
df_test = pd.read_csv(f"{OUTPUT_FOLDER}/wedfri_annotated_test.csv", parse_dates=['Timestamp'], date_format="mixed")
df_test

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,15431,172.16.0.1-192.168.10.50-54018-80-6,172.16.0.1,54018,192.168.10.50,80,6,2017-07-05 13:06:06.833561,41,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.951220e+05,48780.487805,41.000000,0.000000,41.0,41.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,24390.243902,24390.243902,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1032238673,Slowloris
1,15432,172.16.0.1-192.168.10.50-54126-80-6,172.16.0.1,54126,192.168.10.50,80,6,2017-07-05 13:06:06.833566,9,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,8.888889e+05,222222.222222,9.000000,0.000000,9.0,9.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,111111.111111,111111.111111,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1024233804,Slowloris
2,15433,172.16.0.1-192.168.10.50-54140-80-6,172.16.0.1,54140,192.168.10.50,80,6,2017-07-05 13:06:06.833595,29,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,2.758621e+05,68965.517241,29.000000,0.000000,29.0,29.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,34482.758621,34482.758621,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1023232173,Slowloris
3,15434,172.16.0.1-192.168.10.50-54178-80-6,172.16.0.1,54178,192.168.10.50,80,6,2017-07-05 13:06:06.833606,58,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.379310e+05,34482.758621,58.000000,0.000000,58.0,58.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,17241.379310,17241.379310,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1021233337,Slowloris
4,15435,172.16.0.1-192.168.10.50-54146-80-6,172.16.0.1,54146,192.168.10.50,80,6,2017-07-05 13:06:06.833601,17,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,4.705882e+05,117647.058824,17.000000,0.000000,17.0,17.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0

## Creating PCAP

In [3]:
flow_packets = []
with open(f"{OUTPUT_FOLDER}/wednesday-friday-24-april-2025.pcap_Flow.csv_index.txt") as file:
    for line in file:
        indexes = line.strip().split(",")
        if len(indexes[-1]) == 0:
            indexes = indexes[:-1]
        indexes = [int(x) for x in indexes]
        flow_packets.append(indexes)
flow_packets

[[11457, 11463],
 [11464, 11480],
 [11487, 11488],
 [11500, 11523],
 [11570, 11588],
 [11616, 11656],
 [11785, 11898],
 [11962, 11998],
 [12012, 12127],
 [12133, 12290],
 [12315, 12352],
 [12354, 12383],
 [12527, 12536],
 [12873, 12905],
 [13534, 13592],
 [13616, 13998],
 [58,
  60,
  62,
  63,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  1538,
  1540,
  1542,
  1543,
  1545,
  1546,
  1548,
  1549,
  1551,
  1552,
  1554,
  9813,
  9815,
  9817,
  9818,
  9820,
  9821,
  9823,
  9824,
  9826,
  9827,
  9828,
  11640,
  11643,
  11645,
  11646,
  11648,
  11649,
  11651,
  11652,
  11653,
  11654,
  11655],
 [382, 12130, 389, 12144],
 [107, 12473, 109, 12505],
 [428,
  429,
  430,
  431,
  432,
  433,
  434,
  435,
  436,
  437,
  438,
  1892,
  10334,
  10335,
  10525,
  10526,
  10527,
  11024,
  11117,
  11118,
  11266,
  11267,
  11268,
  11269,
  11270,
  11271,
  11272,
  11273,
  11274,
  11275,
  11276,
  11277,
  11301,
  11302,
  11303,
  11304,
  11305,
  11306,
  11307,
  11

In [5]:
df_test = pd.read_csv(f"{OUTPUT_FOLDER}/wedfri_annotated_test.csv", parse_dates=['Timestamp'], date_format="mixed", index_col=0)
test_flow_indexes = list(df_test.index)

In [6]:
df_train = pd.read_csv(f"{OUTPUT_FOLDER}/wedfri_annotated_train.csv", parse_dates=['Timestamp'], date_format="mixed", index_col=0)
train_flow_indexes = list(df_train.index)

In [7]:
import itertools

test_packet_idx_unflattened = [flow_packets[i] for i in test_flow_indexes]
test_packet_idx = list(itertools.chain.from_iterable(test_packet_idx_unflattened))
test_packet_idx_set = set(test_packet_idx)

train_packet_idx_unflattened = [flow_packets[i] for i in train_flow_indexes]
train_packet_idx = list(itertools.chain.from_iterable(train_packet_idx_unflattened))
train_packet_idx_set = set(train_packet_idx)

In [13]:
df_test.duplicated().any()

False

In [14]:
df_train.duplicated().any()

False

In [15]:
len(test_packet_idx) == len(test_packet_idx_set)

True

In [16]:
len(train_packet_idx) == len(train_packet_idx_set)

True

In [17]:
import gc
from scapy.all import PcapReader, wrpcap

with PcapReader(PCAP_INPUT) as pr:
    pcap_reader: PcapReader = pr

    train_out_packets = []
    test_out_packets = []

    train_cnt_pcap = 0
    test_cnt_pcap = 0
    i = 0
    while True:
        if i % 100_000 == 0:
            print(i)

        if len(train_out_packets) >= 500_000:
            print("SAVING TRAINING SET...")
            wrpcap(f"/mnt/extra/datasets/pcap/wedfri_train_{train_cnt_pcap}.pcap", train_out_packets)
            del train_out_packets
            gc.collect()
            train_out_packets = []
            train_cnt_pcap += 1

        if len(test_out_packets) >= 500_000:
            print("SAVING TESTING SET...")
            wrpcap(f"/mnt/extra/datasets/pcap/wedfri_test_{test_cnt_pcap}.pcap", test_out_packets)
            del test_out_packets
            gc.collect()
            test_out_packets = []
            test_cnt_pcap += 1

        try:
            packet = pcap_reader.read_packet()
        except:
            print(f"{i} packets have been iterated")
            break
        
        cnt_append = 0
        if i in train_packet_idx_set: # "not in" for training set
            train_out_packets.append(packet)
            cnt_append += 1
        
        if i in test_packet_idx_set: # "not in" for training set
            test_out_packets.append(packet)
            cnt_append += 1
        
        assert cnt_append < 2

        i += 1

if len(train_out_packets) > 0:
    print("SAVING TRAINING SET...")
    wrpcap(f"/mnt/extra/datasets/pcap/wedfri_train_{train_cnt_pcap}.pcap", train_out_packets)

if len(test_out_packets) > 0:
    print("SAVING TEST SET...")
    wrpcap(f"/mnt/extra/datasets/pcap/wedfri_test_{test_cnt_pcap}.pcap", test_out_packets)

print("DONE")

0
100000
200000
300000
400000
500000
SAVING TRAINING SET...
600000
700000
800000
900000
1000000
SAVING TRAINING SET...
1100000
1200000
1300000
1400000
1500000
SAVING TRAINING SET...
1600000
1700000
1800000
1900000
2000000
SAVING TRAINING SET...
2100000
2200000
2300000
2400000
2500000
SAVING TRAINING SET...
2600000
2700000
2800000
2900000
3000000
SAVING TRAINING SET...
3100000
3200000
3300000
3400000
3500000
SAVING TRAINING SET...
3600000
3700000
3800000
3900000
4000000
SAVING TRAINING SET...
4100000
4200000
4300000
4400000
4500000
SAVING TRAINING SET...
4600000
4700000
4800000
4900000
5000000
SAVING TRAINING SET...
5100000
5200000
5300000
5400000
5500000
SAVING TRAINING SET...
5600000
5700000
5800000
5900000
6000000
SAVING TRAINING SET...
6100000
6200000
6300000
6400000
6500000
6600000
SAVING TRAINING SET...
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900

In [21]:
import os

print(f"Generated {train_cnt_pcap} training set subpcaps")
train_command = f'mergecap -F pcap -w /mnt/extra/datasets/pcap/wedfri_train.pcap'
for i in range(train_cnt_pcap+1):
    train_command += f' /mnt/extra/datasets/pcap/wedfri_train_{i}.pcap'

print(f"Generated {test_cnt_pcap} testing set subpcaps")
test_command = f'mergecap -F pcap -w /mnt/extra/datasets/pcap/wedfri_test.pcap'
for i in range(test_cnt_pcap+1):
    test_command += f' /mnt/extra/datasets/pcap/wedfri_test_{i}.pcap'

print(train_command)
print(test_command)

os.system(train_command)
os.system(test_command)

Generated 35 training set subpcaps
Generated 4 testing set subpcaps
mergecap -F pcap -w /mnt/extra/datasets/pcap/wedfri_train.pcap /mnt/extra/datasets/pcap/wedfri_train_0.pcap /mnt/extra/datasets/pcap/wedfri_train_1.pcap /mnt/extra/datasets/pcap/wedfri_train_2.pcap /mnt/extra/datasets/pcap/wedfri_train_3.pcap /mnt/extra/datasets/pcap/wedfri_train_4.pcap /mnt/extra/datasets/pcap/wedfri_train_5.pcap /mnt/extra/datasets/pcap/wedfri_train_6.pcap /mnt/extra/datasets/pcap/wedfri_train_7.pcap /mnt/extra/datasets/pcap/wedfri_train_8.pcap /mnt/extra/datasets/pcap/wedfri_train_9.pcap /mnt/extra/datasets/pcap/wedfri_train_10.pcap /mnt/extra/datasets/pcap/wedfri_train_11.pcap /mnt/extra/datasets/pcap/wedfri_train_12.pcap /mnt/extra/datasets/pcap/wedfri_train_13.pcap /mnt/extra/datasets/pcap/wedfri_train_14.pcap /mnt/extra/datasets/pcap/wedfri_train_15.pcap /mnt/extra/datasets/pcap/wedfri_train_16.pcap /mnt/extra/datasets/pcap/wedfri_train_17.pcap /mnt/extra/datasets/pcap/wedfri_train_18.pcap /mnt/

0

# Merge with Flink Results
## Train

In [26]:
FLINK_TRAIN_FILE = "./flink-equivalency/flink_wedfri_train.csv"

In [27]:
flink_train = pd.read_csv(FLINK_TRAIN_FILE, parse_dates=['Timestamp'], date_format="mixed")
flink_train

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time
0,b438537d-bc0d-479e-8586-d63f936d1cc0,192.168.10.17,49453,192.168.10.3,389,6,2017-07-05 11:42:48.570260,527.0,12,7,172.0,270.0,79.0,0.0,14.333333,30.323508,135.0,0.0,38.571429,65.873255,8.387097e+05,36053.130930,29.277778,25.547661,73.0,0.0,480.0,43.636364,39.558242,109.0,1.0,450.0,75.000000,89.225557,237.0,1.0,4,2,0,0,0,1,400,228,22770.398482,13282.732448,0.0,135.0,23.263158,46.396650,2152.649123,2,4,1,6,17,0,0,0,0.583333,23.263158,14.333333,38.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,4,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6,3,9,0
1,ebc7428b-544d-4d1d-b0ea-f7943ba6e8c3,192.168.10.17,46124,192.168.10.3,88,6,2017-07-05 11:42:48.583448,1194.0,11,9,3150.0,3150.0,1575.0,0.0,286.363636,607.569959,1575.0,0.0,350.000000,694.509719,5.276382e+06,16750.418760,62.842105,182.412007,810.0,1.0,1096.0,109.600000,284.233550,915.0,1.0,1156.0,144.500000,272.931179,810.0,1.0,2,2,0,0,0,1,368,292,9212.730318,7537.688442,0.0,1575.0,315.000000,631.214702,398432.000000,2,4,1,4,18,0,0,0,0.818182,315.000000,286.363636,350.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,3,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,5,4,9,0
2,51152b73-7a17-45ba-9d8d-fc5cdbf12560,192.168.10.17,49454,192.168.10.3,389,6,2017-07-05 11:42:48.571557,15185.0,16,12,3452.0,6423.0,1313.0,0.0,215.750000,436.408601,2884.0,0.0,535.250000,937.110271,6.503128e+05,1843.924926,562.407407,2565.816773,13391.0,0.0,15156.0,1010.400000,3430.037293,13391.0,2.0,15112.0,1373.818182,4176.449588,13961.0,3.0,10,8,0,0,0,1,528,388,1053.671386,790.253540,0.0,2884.0,352.678571,699.648010,489507.337302,0,4,1,18,26,0,0,0,0.750000,352.678571,215.750000,535.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,10,9,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,8,4,12,0
3,b641355f-2e6a-4b71-804b-fae52ca8c2ec,192.168.10.17,46126,192.168.10.3,88,6,2017-07-05 11:42:48.627714,1186.0,10,9,3150.0,3152.0,1575.0,0.0,315.000000,664.078309,1576.0,0.0,350.222222,694.950678,5.313659e+06,16020.236088,65.888889,183.078861,794.0,0.0,1093.0,121.444444,297.034557,910.0,1.0,1139.0,142.375000,266.191627,794.0,0.0,2,2,0,0,0,1,336,292,8431.703204,7588.532884,0.0,1576.0,331.684211,659.904375,435473.783626,2,4,1,4,17,0,0,0,0.900000,331.684211,315.000000,350.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,2,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6,4,10,0
4,759d98b5-2186-4bb4-96fb-79c24ea18114,192.168.10.17,49460,192.168.10.3,389,6,2017-07-05 11:42:49.114734,483.0,12,7,172.0,270.0,79.0,0.0,14.333333,30.323508,135.0,0.0,38.571429,

In [28]:
flink_train_keys = flink_train[distinguishing_keys]
has_duplicate = (flink_train_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

flink_train = flink_train[~flink_train_keys.duplicated(keep=False)]
flink_train

Has Duplicates? False


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time
0,b438537d-bc0d-479e-8586-d63f936d1cc0,192.168.10.17,49453,192.168.10.3,389,6,2017-07-05 11:42:48.570260,527.0,12,7,172.0,270.0,79.0,0.0,14.333333,30.323508,135.0,0.0,38.571429,65.873255,8.387097e+05,36053.130930,29.277778,25.547661,73.0,0.0,480.0,43.636364,39.558242,109.0,1.0,450.0,75.000000,89.225557,237.0,1.0,4,2,0,0,0,1,400,228,22770.398482,13282.732448,0.0,135.0,23.263158,46.396650,2152.649123,2,4,1,6,17,0,0,0,0.583333,23.263158,14.333333,38.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,4,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6,3,9,0
1,ebc7428b-544d-4d1d-b0ea-f7943ba6e8c3,192.168.10.17,46124,192.168.10.3,88,6,2017-07-05 11:42:48.583448,1194.0,11,9,3150.0,3150.0,1575.0,0.0,286.363636,607.569959,1575.0,0.0,350.000000,694.509719,5.276382e+06,16750.418760,62.842105,182.412007,810.0,1.0,1096.0,109.600000,284.233550,915.0,1.0,1156.0,144.500000,272.931179,810.0,1.0,2,2,0,0,0,1,368,292,9212.730318,7537.688442,0.0,1575.0,315.000000,631.214702,398432.000000,2,4,1,4,18,0,0,0,0.818182,315.000000,286.363636,350.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,3,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,5,4,9,0
2,51152b73-7a17-45ba-9d8d-fc5cdbf12560,192.168.10.17,49454,192.168.10.3,389,6,2017-07-05 11:42:48.571557,15185.0,16,12,3452.0,6423.0,1313.0,0.0,215.750000,436.408601,2884.0,0.0,535.250000,937.110271,6.503128e+05,1843.924926,562.407407,2565.816773,13391.0,0.0,15156.0,1010.400000,3430.037293,13391.0,2.0,15112.0,1373.818182,4176.449588,13961.0,3.0,10,8,0,0,0,1,528,388,1053.671386,790.253540,0.0,2884.0,352.678571,699.648010,489507.337302,0,4,1,18,26,0,0,0,0.750000,352.678571,215.750000,535.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,10,9,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,8,4,12,0
3,b641355f-2e6a-4b71-804b-fae52ca8c2ec,192.168.10.17,46126,192.168.10.3,88,6,2017-07-05 11:42:48.627714,1186.0,10,9,3150.0,3152.0,1575.0,0.0,315.000000,664.078309,1576.0,0.0,350.222222,694.950678,5.313659e+06,16020.236088,65.888889,183.078861,794.0,0.0,1093.0,121.444444,297.034557,910.0,1.0,1139.0,142.375000,266.191627,794.0,0.0,2,2,0,0,0,1,336,292,8431.703204,7588.532884,0.0,1576.0,331.684211,659.904375,435473.783626,2,4,1,4,17,0,0,0,0.900000,331.684211,315.000000,350.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,2,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6,4,10,0
4,759d98b5-2186-4bb4-96fb-79c24ea18114,192.168.10.17,49460,192.168.10.3,389,6,2017-07-05 11:42:49.114734,483.0,12,7,172.0,270.0,79.0,0.0,14.333333,30.323508,135.0,0.0,38.571429,

In [29]:
df_train_keys = df_train[distinguishing_keys + ['Label']]
has_duplicate = (df_train_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

df_train = df_train[~df_train_keys.duplicated(keep=False)]
df_train

Has Duplicates? False


,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
0,0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,Benign,2017-07-05 11:43:34.971591
1,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,Benign,2017-07-05 11:43:35.672523
2,2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,Benign,2017-07-05 11:43:36.565099
3,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,190117,Benign,2017-07-05 11:43:37.266

In [30]:
left_merge = flink_train_keys.reset_index().merge(df_train_keys, on=distinguishing_keys, how='left', indicator=True).set_index('index')
left_merge

,Src IP,Src Port,Dst IP,Dst Port,Timestamp,Label,_merge
index,,,,,,,
0,192.168.10.17,49453,192.168.10.3,389,2017-07-05 11:42:48.570260,Benign,both
1,192.168.10.17,46124,192.168.10.3,88,2017-07-05 11:42:48.583448,Benign,both
2,192.168.10.17,49454,192.168.10.3,389,2017-07-05 11:42:48.571557,Benign,both
3,192.168.10.17,46126,192.168.10.3,88,2017-07-05 11:42:48.627714,Benign,both
4,192.168.10.17,49460,192.168.10.3,389,2017-07-05 11:42:49.114734,Benign,both
...,...,...,...,...,...,...,...
616906,172.16.0.1,52646,192.168.10.50,8093,2017-07-07 17:55:45.861240,Portscan,both
616907,172.16.0.1,40874,192.168.10.50,5907,2017-07-07 17:55:45.861288,Portscan,both
616908,172.16.0.1,54672,192.168.10.50,52869,2017-07-07 17:55:45.861383,Portscan,both


In [31]:
flink_train_ignored = left_merge[left_merge['_merge'] == 'left_only']
flink_train_ignored_idx = flink_train_ignored.index
flink_train = flink_train.drop(index=flink_train_ignored_idx)
flink_train

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time
0,b438537d-bc0d-479e-8586-d63f936d1cc0,192.168.10.17,49453,192.168.10.3,389,6,2017-07-05 11:42:48.570260,527.0,12,7,172.0,270.0,79.0,0.0,14.333333,30.323508,135.0,0.0,38.571429,65.873255,8.387097e+05,36053.130930,29.277778,25.547661,73.0,0.0,480.0,43.636364,39.558242,109.0,1.0,450.0,75.000000,89.225557,237.0,1.0,4,2,0,0,0,1,400,228,22770.398482,13282.732448,0.0,135.0,23.263158,46.396650,2152.649123,2,4,1,6,17,0,0,0,0.583333,23.263158,14.333333,38.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,4,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6,3,9,0
1,ebc7428b-544d-4d1d-b0ea-f7943ba6e8c3,192.168.10.17,46124,192.168.10.3,88,6,2017-07-05 11:42:48.583448,1194.0,11,9,3150.0,3150.0,1575.0,0.0,286.363636,607.569959,1575.0,0.0,350.000000,694.509719,5.276382e+06,16750.418760,62.842105,182.412007,810.0,1.0,1096.0,109.600000,284.233550,915.0,1.0,1156.0,144.500000,272.931179,810.0,1.0,2,2,0,0,0,1,368,292,9212.730318,7537.688442,0.0,1575.0,315.000000,631.214702,398432.000000,2,4,1,4,18,0,0,0,0.818182,315.000000,286.363636,350.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,3,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,5,4,9,0
2,51152b73-7a17-45ba-9d8d-fc5cdbf12560,192.168.10.17,49454,192.168.10.3,389,6,2017-07-05 11:42:48.571557,15185.0,16,12,3452.0,6423.0,1313.0,0.0,215.750000,436.408601,2884.0,0.0,535.250000,937.110271,6.503128e+05,1843.924926,562.407407,2565.816773,13391.0,0.0,15156.0,1010.400000,3430.037293,13391.0,2.0,15112.0,1373.818182,4176.449588,13961.0,3.0,10,8,0,0,0,1,528,388,1053.671386,790.253540,0.0,2884.0,352.678571,699.648010,489507.337302,0,4,1,18,26,0,0,0,0.750000,352.678571,215.750000,535.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,10,9,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,8,4,12,0
3,b641355f-2e6a-4b71-804b-fae52ca8c2ec,192.168.10.17,46126,192.168.10.3,88,6,2017-07-05 11:42:48.627714,1186.0,10,9,3150.0,3152.0,1575.0,0.0,315.000000,664.078309,1576.0,0.0,350.222222,694.950678,5.313659e+06,16020.236088,65.888889,183.078861,794.0,0.0,1093.0,121.444444,297.034557,910.0,1.0,1139.0,142.375000,266.191627,794.0,0.0,2,2,0,0,0,1,336,292,8431.703204,7588.532884,0.0,1576.0,331.684211,659.904375,435473.783626,2,4,1,4,17,0,0,0,0.900000,331.684211,315.000000,350.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,2,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6,4,10,0
4,759d98b5-2186-4bb4-96fb-79c24ea18114,192.168.10.17,49460,192.168.10.3,389,6,2017-07-05 11:42:49.114734,483.0,12,7,172.0,270.0,79.0,0.0,14.333333,30.323508,135.0,0.0,38.571429,

In [33]:
flink_train['Label'] = left_merge[left_merge['_merge'] == "both"]['Label']
flink_train

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,b438537d-bc0d-479e-8586-d63f936d1cc0,192.168.10.17,49453,192.168.10.3,389,6,2017-07-05 11:42:48.570260,527.0,12,7,172.0,270.0,79.0,0.0,14.333333,30.323508,135.0,0.0,38.571429,65.873255,8.387097e+05,36053.130930,29.277778,25.547661,73.0,0.0,480.0,43.636364,39.558242,109.0,1.0,450.0,75.000000,89.225557,237.0,1.0,4,2,0,0,0,1,400,228,22770.398482,13282.732448,0.0,135.0,23.263158,46.396650,2152.649123,2,4,1,6,17,0,0,0,0.583333,23.263158,14.333333,38.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,4,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6,3,9,0,Benign
1,ebc7428b-544d-4d1d-b0ea-f7943ba6e8c3,192.168.10.17,46124,192.168.10.3,88,6,2017-07-05 11:42:48.583448,1194.0,11,9,3150.0,3150.0,1575.0,0.0,286.363636,607.569959,1575.0,0.0,350.000000,694.509719,5.276382e+06,16750.418760,62.842105,182.412007,810.0,1.0,1096.0,109.600000,284.233550,915.0,1.0,1156.0,144.500000,272.931179,810.0,1.0,2,2,0,0,0,1,368,292,9212.730318,7537.688442,0.0,1575.0,315.000000,631.214702,398432.000000,2,4,1,4,18,0,0,0,0.818182,315.000000,286.363636,350.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,3,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,5,4,9,0,Benign
2,51152b73-7a17-45ba-9d8d-fc5cdbf12560,192.168.10.17,49454,192.168.10.3,389,6,2017-07-05 11:42:48.571557,15185.0,16,12,3452.0,6423.0,1313.0,0.0,215.750000,436.408601,2884.0,0.0,535.250000,937.110271,6.503128e+05,1843.924926,562.407407,2565.816773,13391.0,0.0,15156.0,1010.400000,3430.037293,13391.0,2.0,15112.0,1373.818182,4176.449588,13961.0,3.0,10,8,0,0,0,1,528,388,1053.671386,790.253540,0.0,2884.0,352.678571,699.648010,489507.337302,0,4,1,18,26,0,0,0,0.750000,352.678571,215.750000,535.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,10,9,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,8,4,12,0,Benign
3,b641355f-2e6a-4b71-804b-fae52ca8c2ec,192.168.10.17,46126,192.168.10.3,88,6,2017-07-05 11:42:48.627714,1186.0,10,9,3150.0,3152.0,1575.0,0.0,315.000000,664.078309,1576.0,0.0,350.222222,694.950678,5.313659e+06,16020.236088,65.888889,183.078861,794.0,0.0,1093.0,121.444444,297.034557,910.0,1.0,1139.0,142.375000,266.191627,794.0,0.0,2,2,0,0,0,1,336,292,8431.703204,7588.532884,0.0,1576.0,331.684211,659.904375,435473.783626,2,4,1,4,17,0,0,0,0.900000,331.684211,315.000000,350.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,8192,2,2,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,6,4,10,0,Benign
4,759d98b5-2186-4bb4-96fb-79c24ea18114,192.168.10.17,49460,192.168.10.3,389,6,2017-07-05 11:42:49.114734,483.0,12,7,172.0,270.0,79.0,0.0,14.333

In [35]:
flink_train['Label'].value_counts(dropna=False)

Label
Benign       484417
Portscan     127254
Slowloris      4563
Name: count, dtype: int64

In [37]:
flink_train.reset_index().to_csv(f"{OUTPUT_FOLDER}/wedfri_train_optimized.csv", index=False)

## Test

In [38]:
FLINK_TEST_FILE = "./flink-equivalency/flink_wedfri_test.csv"

In [39]:
flink_test = pd.read_csv(FLINK_TEST_FILE, parse_dates=['Timestamp'], date_format="mixed")
flink_test

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time
0,5a490cba-63b4-41c6-94a9-feb15fc1baec,192.168.10.14,51104,93.184.216.180,443,6,2017-07-05 13:05:56.267121,85742.0,10,6,527.0,5098.0,192.0,0.0,52.700000,77.209743,2920.0,0.0,849.666667,1149.916461,6.560379e+04,186.606331,5716.133333,9805.393821,25870.0,4.0,85742.0,9526.888889,11978.634714,25927.0,4.0,62651.0,12530.200000,16515.098825,36090.0,49.0,4,2,0,0,1,0,212,132,116.628957,69.977374,0.0,2920.0,351.562500,776.617707,603135.062500,1,2,1,6,15,0,0,0,0.600000,351.562500,52.700000,849.666667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,65535,4,4,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,80fa64b1-f3fc-4d69-8ef7-0446dbf076d6,192.168.10.51,47817,162.213.33.48,443,6,2017-07-05 13:05:56.893194,1038445.0,10,12,703.0,3950.0,267.0,0.0,70.300000,100.477803,1448.0,0.0,329.166667,553.194826,4.480738e+03,21.185523,49449.761905,71821.015885,292001.0,4.0,943512.0,104834.666667,119618.922107,387218.0,217.0,942763.0,85705.727273,92551.804558,292001.0,4.0,4,3,0,0,0,0,328,392,9.629783,11.555740,0.0,1448.0,211.500000,426.650939,182031.023810,2,2,0,7,21,0,0,0,1.200000,211.500000,70.300000,329.166667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,29200,28960,4,5,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,0057811c-b05a-4b9d-b3ba-3bae3117ec04,192.168.10.14,51114,178.255.83.1,80,6,2017-07-05 13:05:57.976332,70672.0,5,5,429.0,858.0,429.0,0.0,85.800000,191.854632,858.0,0.0,171.600000,383.709265,1.821089e+04,141.498755,7852.444444,11565.016873,23364.0,4.0,47565.0,11891.250000,13589.575573,23893.0,109.0,47308.0,11827.000000,13368.062612,23512.0,4.0,1,1,0,0,0,0,112,112,70.749377,70.749377,0.0,858.0,128.700000,289.552931,83840.900000,2,2,0,2,9,0,0,0,1.000000,128.700000,85.800000,171.600000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,14600,1,1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0
3,0becccf2-5d8b-469d-b301-551b1141d21d,192.168.10.14,51122,63.251.85.32,443,6,2017-07-05 13:05:58.340753,217279.0,9,11,2246.0,6435.0,1701.0,0.0,249.555556,558.017946,1877.0,0.0,585.000000,785.136167,3.995324e+04,92.047552,11435.736842,15722.419869,50457.0,1.0,189169.0,23646.125000,26965.015766,81777.0,375.0,189013.0,18901.300000,30703.415948,97524.0,1.0,3,6,0,0,0,0,192,224,41.421398,50.626153,0.0,1877.0,434.050000,696.323010,484865.734211,2,2,0,9,19,0,0,0,1.222222,434.050000,249.555556,585.000000,0.0,0.0,0.0,4430.0,4.0,1.671698e+07,0.0,0.0,0.0,0.0,8192,8190,3,7,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0
4,805693ca-6663-4406-90a8-fb921d937b44,192.168.10.14,51121,63.251.85.

In [40]:
flink_test_keys = flink_test[distinguishing_keys]
has_duplicate = (flink_test_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

flink_test = flink_test[~flink_test_keys.duplicated(keep=False)]
flink_test

Has Duplicates? False


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time
0,5a490cba-63b4-41c6-94a9-feb15fc1baec,192.168.10.14,51104,93.184.216.180,443,6,2017-07-05 13:05:56.267121,85742.0,10,6,527.0,5098.0,192.0,0.0,52.700000,77.209743,2920.0,0.0,849.666667,1149.916461,6.560379e+04,186.606331,5716.133333,9805.393821,25870.0,4.0,85742.0,9526.888889,11978.634714,25927.0,4.0,62651.0,12530.200000,16515.098825,36090.0,49.0,4,2,0,0,1,0,212,132,116.628957,69.977374,0.0,2920.0,351.562500,776.617707,603135.062500,1,2,1,6,15,0,0,0,0.600000,351.562500,52.700000,849.666667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,65535,4,4,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,80fa64b1-f3fc-4d69-8ef7-0446dbf076d6,192.168.10.51,47817,162.213.33.48,443,6,2017-07-05 13:05:56.893194,1038445.0,10,12,703.0,3950.0,267.0,0.0,70.300000,100.477803,1448.0,0.0,329.166667,553.194826,4.480738e+03,21.185523,49449.761905,71821.015885,292001.0,4.0,943512.0,104834.666667,119618.922107,387218.0,217.0,942763.0,85705.727273,92551.804558,292001.0,4.0,4,3,0,0,0,0,328,392,9.629783,11.555740,0.0,1448.0,211.500000,426.650939,182031.023810,2,2,0,7,21,0,0,0,1.200000,211.500000,70.300000,329.166667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,29200,28960,4,5,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,0057811c-b05a-4b9d-b3ba-3bae3117ec04,192.168.10.14,51114,178.255.83.1,80,6,2017-07-05 13:05:57.976332,70672.0,5,5,429.0,858.0,429.0,0.0,85.800000,191.854632,858.0,0.0,171.600000,383.709265,1.821089e+04,141.498755,7852.444444,11565.016873,23364.0,4.0,47565.0,11891.250000,13589.575573,23893.0,109.0,47308.0,11827.000000,13368.062612,23512.0,4.0,1,1,0,0,0,0,112,112,70.749377,70.749377,0.0,858.0,128.700000,289.552931,83840.900000,2,2,0,2,9,0,0,0,1.000000,128.700000,85.800000,171.600000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,14600,1,1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0
3,0becccf2-5d8b-469d-b301-551b1141d21d,192.168.10.14,51122,63.251.85.32,443,6,2017-07-05 13:05:58.340753,217279.0,9,11,2246.0,6435.0,1701.0,0.0,249.555556,558.017946,1877.0,0.0,585.000000,785.136167,3.995324e+04,92.047552,11435.736842,15722.419869,50457.0,1.0,189169.0,23646.125000,26965.015766,81777.0,375.0,189013.0,18901.300000,30703.415948,97524.0,1.0,3,6,0,0,0,0,192,224,41.421398,50.626153,0.0,1877.0,434.050000,696.323010,484865.734211,2,2,0,9,19,0,0,0,1.222222,434.050000,249.555556,585.000000,0.0,0.0,0.0,4430.0,4.0,1.671698e+07,0.0,0.0,0.0,0.0,8192,8190,3,7,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0
4,805693ca-6663-4406-90a8-fb921d937b44,192.168.10.14,51121,63.251.85.

In [43]:
df_test_keys = df_test[distinguishing_keys + ['Label']]
has_duplicate = (df_test_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

df_test = df_test[~df_test_keys.duplicated(keep=False)]
df_test

Has Duplicates? False


,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,15431,172.16.0.1-192.168.10.50-54018-80-6,172.16.0.1,54018,192.168.10.50,80,6,2017-07-05 13:06:06.833561,41,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.951220e+05,48780.487805,41.000000,0.000000,41.0,41.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,24390.243902,24390.243902,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1032238673,Slowloris
1,15432,172.16.0.1-192.168.10.50-54126-80-6,172.16.0.1,54126,192.168.10.50,80,6,2017-07-05 13:06:06.833566,9,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,8.888889e+05,222222.222222,9.000000,0.000000,9.0,9.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,111111.111111,111111.111111,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1024233804,Slowloris
2,15433,172.16.0.1-192.168.10.50-54140-80-6,172.16.0.1,54140,192.168.10.50,80,6,2017-07-05 13:06:06.833595,29,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,2.758621e+05,68965.517241,29.000000,0.000000,29.0,29.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,34482.758621,34482.758621,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1023232173,Slowloris
3,15434,172.16.0.1-192.168.10.50-54178-80-6,172.16.0.1,54178,192.168.10.50,80,6,2017-07-05 13:06:06.833606,58,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.379310e+05,34482.758621,58.000000,0.000000,58.0,58.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,17241.379310,17241.379310,0.0,8.0,4.000000,5.656854,3.200000e+01,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,229,235,1,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,1021233337,Slowloris
4,15435,172.16.0.1-192.168.10.50-54146-80-6,172.16.0.1,54146,192.168.10.50,80,6,2017-07-05 13:06:06.833601,17,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,4.705882e+05,117647.058824,17.000000,0.000000,17.0,17.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0

In [44]:
left_merge = flink_test_keys.reset_index().merge(df_test_keys, on=distinguishing_keys, how='left', indicator=True).set_index('index')
left_merge

,Src IP,Src Port,Dst IP,Dst Port,Timestamp,Label,_merge
index,,,,,,,
0,192.168.10.14,51104,93.184.216.180,443,2017-07-05 13:05:56.267121,Benign,both
1,192.168.10.51,47817,162.213.33.48,443,2017-07-05 13:05:56.893194,Benign,both
2,192.168.10.14,51114,178.255.83.1,80,2017-07-05 13:05:57.976332,Benign,both
3,192.168.10.14,51122,63.251.85.32,443,2017-07-05 13:05:58.340753,Benign,both
4,192.168.10.14,51121,63.251.85.32,443,2017-07-05 13:05:58.340662,Benign,both
...,...,...,...,...,...,...,...
154073,192.168.10.50,42910,192.168.10.3,389,2017-07-07 20:02:38.387716,Benign,both
154074,192.168.10.8,51617,192.168.10.3,53,2017-07-07 20:02:38.574765,Benign,both
154075,192.168.10.8,10414,192.168.10.3,445,2017-07-07 20:02:38.575222,Benign,both


In [45]:
flink_test_ignored = left_merge[left_merge['_merge'] == 'left_only']
flink_test_ignored_idx = flink_test_ignored.index
flink_test = flink_test.drop(index=flink_test_ignored_idx)
flink_test

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time
0,5a490cba-63b4-41c6-94a9-feb15fc1baec,192.168.10.14,51104,93.184.216.180,443,6,2017-07-05 13:05:56.267121,85742.0,10,6,527.0,5098.0,192.0,0.0,52.700000,77.209743,2920.0,0.0,849.666667,1149.916461,6.560379e+04,186.606331,5716.133333,9805.393821,25870.0,4.0,85742.0,9526.888889,11978.634714,25927.0,4.0,62651.0,12530.200000,16515.098825,36090.0,49.0,4,2,0,0,1,0,212,132,116.628957,69.977374,0.0,2920.0,351.562500,776.617707,603135.062500,1,2,1,6,15,0,0,0,0.600000,351.562500,52.700000,849.666667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,65535,4,4,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,80fa64b1-f3fc-4d69-8ef7-0446dbf076d6,192.168.10.51,47817,162.213.33.48,443,6,2017-07-05 13:05:56.893194,1038445.0,10,12,703.0,3950.0,267.0,0.0,70.300000,100.477803,1448.0,0.0,329.166667,553.194826,4.480738e+03,21.185523,49449.761905,71821.015885,292001.0,4.0,943512.0,104834.666667,119618.922107,387218.0,217.0,942763.0,85705.727273,92551.804558,292001.0,4.0,4,3,0,0,0,0,328,392,9.629783,11.555740,0.0,1448.0,211.500000,426.650939,182031.023810,2,2,0,7,21,0,0,0,1.200000,211.500000,70.300000,329.166667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,29200,28960,4,5,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,0057811c-b05a-4b9d-b3ba-3bae3117ec04,192.168.10.14,51114,178.255.83.1,80,6,2017-07-05 13:05:57.976332,70672.0,5,5,429.0,858.0,429.0,0.0,85.800000,191.854632,858.0,0.0,171.600000,383.709265,1.821089e+04,141.498755,7852.444444,11565.016873,23364.0,4.0,47565.0,11891.250000,13589.575573,23893.0,109.0,47308.0,11827.000000,13368.062612,23512.0,4.0,1,1,0,0,0,0,112,112,70.749377,70.749377,0.0,858.0,128.700000,289.552931,83840.900000,2,2,0,2,9,0,0,0,1.000000,128.700000,85.800000,171.600000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,14600,1,1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0
3,0becccf2-5d8b-469d-b301-551b1141d21d,192.168.10.14,51122,63.251.85.32,443,6,2017-07-05 13:05:58.340753,217279.0,9,11,2246.0,6435.0,1701.0,0.0,249.555556,558.017946,1877.0,0.0,585.000000,785.136167,3.995324e+04,92.047552,11435.736842,15722.419869,50457.0,1.0,189169.0,23646.125000,26965.015766,81777.0,375.0,189013.0,18901.300000,30703.415948,97524.0,1.0,3,6,0,0,0,0,192,224,41.421398,50.626153,0.0,1877.0,434.050000,696.323010,484865.734211,2,2,0,9,19,0,0,0,1.222222,434.050000,249.555556,585.000000,0.0,0.0,0.0,4430.0,4.0,1.671698e+07,0.0,0.0,0.0,0.0,8192,8190,3,7,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0
4,805693ca-6663-4406-90a8-fb921d937b44,192.168.10.14,51121,63.251.85.

In [46]:
flink_test['Label'] = left_merge[left_merge['_merge'] == "both"]['Label']
flink_test

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,5a490cba-63b4-41c6-94a9-feb15fc1baec,192.168.10.14,51104,93.184.216.180,443,6,2017-07-05 13:05:56.267121,85742.0,10,6,527.0,5098.0,192.0,0.0,52.700000,77.209743,2920.0,0.0,849.666667,1149.916461,6.560379e+04,186.606331,5716.133333,9805.393821,25870.0,4.0,85742.0,9526.888889,11978.634714,25927.0,4.0,62651.0,12530.200000,16515.098825,36090.0,49.0,4,2,0,0,1,0,212,132,116.628957,69.977374,0.0,2920.0,351.562500,776.617707,603135.062500,1,2,1,6,15,0,0,0,0.600000,351.562500,52.700000,849.666667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,65535,4,4,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,Benign
1,80fa64b1-f3fc-4d69-8ef7-0446dbf076d6,192.168.10.51,47817,162.213.33.48,443,6,2017-07-05 13:05:56.893194,1038445.0,10,12,703.0,3950.0,267.0,0.0,70.300000,100.477803,1448.0,0.0,329.166667,553.194826,4.480738e+03,21.185523,49449.761905,71821.015885,292001.0,4.0,943512.0,104834.666667,119618.922107,387218.0,217.0,942763.0,85705.727273,92551.804558,292001.0,4.0,4,3,0,0,0,0,328,392,9.629783,11.555740,0.0,1448.0,211.500000,426.650939,182031.023810,2,2,0,7,21,0,0,0,1.200000,211.500000,70.300000,329.166667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,29200,28960,4,5,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,Benign
2,0057811c-b05a-4b9d-b3ba-3bae3117ec04,192.168.10.14,51114,178.255.83.1,80,6,2017-07-05 13:05:57.976332,70672.0,5,5,429.0,858.0,429.0,0.0,85.800000,191.854632,858.0,0.0,171.600000,383.709265,1.821089e+04,141.498755,7852.444444,11565.016873,23364.0,4.0,47565.0,11891.250000,13589.575573,23893.0,109.0,47308.0,11827.000000,13368.062612,23512.0,4.0,1,1,0,0,0,0,112,112,70.749377,70.749377,0.0,858.0,128.700000,289.552931,83840.900000,2,2,0,2,9,0,0,0,1.000000,128.700000,85.800000,171.600000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,14600,1,1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0,Benign
3,0becccf2-5d8b-469d-b301-551b1141d21d,192.168.10.14,51122,63.251.85.32,443,6,2017-07-05 13:05:58.340753,217279.0,9,11,2246.0,6435.0,1701.0,0.0,249.555556,558.017946,1877.0,0.0,585.000000,785.136167,3.995324e+04,92.047552,11435.736842,15722.419869,50457.0,1.0,189169.0,23646.125000,26965.015766,81777.0,375.0,189013.0,18901.300000,30703.415948,97524.0,1.0,3,6,0,0,0,0,192,224,41.421398,50.626153,0.0,1877.0,434.050000,696.323010,484865.734211,2,2,0,9,19,0,0,0,1.222222,434.050000,249.555556,585.000000,0.0,0.0,0.0,4430.0,4.0,1.671698e+07,0.0,0.0,0.0,0.0,8192,8190,3,7,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0,Benign
4,805693ca-6663-4406-90a8-fb921d937

In [47]:
flink_test['Label'].value_counts(dropna=False)

Label
Benign       120990
Portscan      31786
Slowloris      1139
Name: count, dtype: int64

In [48]:
flink_test.reset_index().to_csv(f"{OUTPUT_FOLDER}/wedfri_test_optimized.csv", index=False)

In [10]:
flink_test = pd.read_csv(f"{OUTPUT_FOLDER}/wedfri_test_optimized.csv", parse_dates=['Timestamp'], date_format="mixed")
flink_test

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,0,5a490cba-63b4-41c6-94a9-feb15fc1baec,192.168.10.14,51104,93.184.216.180,443,6,2017-07-05 13:05:56.267121,85742.0,10,6,527.0,5098.0,192.0,0.0,52.700000,77.209743,2920.0,0.0,849.666667,1149.916461,6.560379e+04,186.606331,5716.133333,9805.393821,25870.0,4.0,85742.0,9526.888889,11978.634714,25927.0,4.0,62651.0,12530.200000,16515.098825,36090.0,49.0,4,2,0,0,1,0,212,132,116.628957,69.977374,0.0,2920.0,351.562500,776.617707,603135.062500,1,2,1,6,15,0,0,0,0.600000,351.562500,52.700000,849.666667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,65535,4,4,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,Benign
1,1,80fa64b1-f3fc-4d69-8ef7-0446dbf076d6,192.168.10.51,47817,162.213.33.48,443,6,2017-07-05 13:05:56.893194,1038445.0,10,12,703.0,3950.0,267.0,0.0,70.300000,100.477803,1448.0,0.0,329.166667,553.194826,4.480738e+03,21.185523,49449.761905,71821.015885,292001.0,4.0,943512.0,104834.666667,119618.922107,387218.0,217.0,942763.0,85705.727273,92551.804558,292001.0,4.0,4,3,0,0,0,0,328,392,9.629783,11.555740,0.0,1448.0,211.500000,426.650939,182031.023810,2,2,0,7,21,0,0,0,1.200000,211.500000,70.300000,329.166667,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,29200,28960,4,5,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,Benign
2,2,0057811c-b05a-4b9d-b3ba-3bae3117ec04,192.168.10.14,51114,178.255.83.1,80,6,2017-07-05 13:05:57.976332,70672.0,5,5,429.0,858.0,429.0,0.0,85.800000,191.854632,858.0,0.0,171.600000,383.709265,1.821089e+04,141.498755,7852.444444,11565.016873,23364.0,4.0,47565.0,11891.250000,13589.575573,23893.0,109.0,47308.0,11827.000000,13368.062612,23512.0,4.0,1,1,0,0,0,0,112,112,70.749377,70.749377,0.0,858.0,128.700000,289.552931,83840.900000,2,2,0,2,9,0,0,0,1.000000,128.700000,85.800000,171.600000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,8192,14600,1,1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0,Benign
3,3,0becccf2-5d8b-469d-b301-551b1141d21d,192.168.10.14,51122,63.251.85.32,443,6,2017-07-05 13:05:58.340753,217279.0,9,11,2246.0,6435.0,1701.0,0.0,249.555556,558.017946,1877.0,0.0,585.000000,785.136167,3.995324e+04,92.047552,11435.736842,15722.419869,50457.0,1.0,189169.0,23646.125000,26965.015766,81777.0,375.0,189013.0,18901.300000,30703.415948,97524.0,1.0,3,6,0,0,0,0,192,224,41.421398,50.626153,0.0,1877.0,434.050000,696.323010,484865.734211,2,2,0,9,19,0,0,0,1.222222,434.050000,249.555556,585.000000,0.0,0.0,0.0,4430.0,4.0,1.671698e+07,0.0,0.0,0.0,0.0,8192,8190,3,7,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0,Benign
4,4,805693ca-6663-440